<a href="https://colab.research.google.com/github/Medynal/Pollution/blob/main/modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import joblib

@st.cache_resource
def load_models():
    aqi_model = joblib.load("aqi_regressor.pkl")
    bucket_model = joblib.load("aqib_classifier.pkl")
    bucket_encoder = joblib.load("bucket_encoder.pkl")
    return aqi_model, bucket_model, bucket_encoder

def show_modelling():

    st.header("AQI Modelling and Prediction")

    df = load_data()
    aqi_model, bucket_model, bucket_encoder = load_models()

    col1, col2 = st.columns(2)

    city = col1.selectbox("City", sorted(df["City"].unique()))
    date = col1.date_input("Date")

    pollutants = [
        'PM2.5','PM10','NO','NO2','NOx',
        'NH3','CO','SO2','O3','Benzene','Toluene','Xylene']

    pollutant_values = {}
    for p in pollutants:
        pollutant_values[p] = col2.number_input(p, min_value=0.0)

    if st.button("Predict AQI"):

        input_df = pd.DataFrame([{
            "City": city,
            "year": date.year,
            "month": date.month,
            "day": date.day,
            **pollutant_values }])

        predicted_aqi = aqi_model.predict(input_df)[0]
        input_df["AQI"] = predicted_aqi

        predicted_bucket = bucket_model.predict(input_df)[0]
        bucket_label = bucket_encoder.inverse_transform([predicted_bucket])[0]

        st.success(f"Predicted AQI: {predicted_aqi:.2f}")
        st.info(f"AQI Category: {bucket_label}")
